In [ ]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.2 MB

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=551ccadf4ea9477c6035968dddc39e6597878ae89d87d781d710b7146f57af72
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gradio as gr
from geopy.geocoders import Nominatim
import folium
import spacy

###Using the model on Web Scraped Data

###Kenya (Property24)

In [ ]:
property24kenyadf = pd.read_csv('/content/Property24Kenya.csv', index_col = None,usecols=list(range(0,12)) )
property24kenyadf.head()

,Property URL,Title,Type,Price,Address,Description,Bedrooms,Bathrooms,Parking spaces,Floor Size,Erf Size,Payment Interval
0,https://www.property24.co.ke/1-bedroom-apartme...,1 Bedroom Apartment / Flat for Sale in Westlands,For Sale,KSh 6 500 000,"Westlands, Westlands, Nairobi",This project adheres to the principle of high-...,1,1,1,,,NaN
1,https://www.property24.co.ke/4-bedroom-townhou...,4 Bedroom Townhouse for Sale in Loresho,For Sale,KSh 38 000 000,"Loresho, Nairobi",LORESHO TOWNHOUSE FOR SALE4 Bedrooms + SQ• mas...,4,4,NaN,,,NaN
2,https://www.property24.co.ke/development-for-s...,3BR Apartments in KILELESHWA on Mandera Road :...,For Sale,From KSh 23 000 000 to KSh 23 000 000,"Kileleshwa, Nairobi",Located in upscale Kileleshwa on Mandera Road ...,NaN,NaN,NaN,,,NaN
3,https://www.property24.co.ke/5-bedroom-townhou...,5 Bedroom Townhouse for Sale in Karen,For Sale,KSh 130 000 000,"Karen Nairobi, Karen, Nairobi",Modern 5 bedroom townhouse for sale in Karen. ...,5,6,4,470 m²,,NaN
4,https://www.property24.co.ke/2-bedroom-apartme...,2 Bedroom Apartment / Flat for Sale in Lavington,For Sale,KSh 9 000 000,"Lavington, Nairobi",City Oasis Homes apartments comprises of 1 and...,2,2,NaN,90 m²,,NaN


In [ ]:
# Convert the text to a matrix of TF-IDF features
kenya_tfidf_vectorizer = TfidfVectorizer()
# Create a combined text column from specific columns
property24kenyadf["combined_text"] = property24kenyadf["Title"].astype(str) + " " + property24kenyadf["Type"].astype(str) + " " + property24kenyadf["Price"].astype(str) + " " + property24kenyadf["Payment Interval"].astype(str) + " " + property24kenyadf["Address"].astype(str) + " " + property24kenyadf["Description"].astype(str) + " " + property24kenyadf["Bedrooms"].astype(str) + " " + property24kenyadf["Bathrooms"].astype(str) + " " + property24kenyadf["Parking spaces"].astype(str) + " " + property24kenyadf["Floor Size"].astype(str) + " " + property24kenyadf["Erf Size"].astype(str)

kenya_tfidf_matrix = kenya_tfidf_vectorizer.fit_transform(property24kenyadf["combined_text"])

###Ethiopia (RealEthio)

In [ ]:
real_ethio_df = pd.read_csv('/content/RealEthioProperties.csv')
real_ethio_df.head()

,Property URL,Title,Type,Price,Address,Description,Property Size,Land Area,Bedrooms,Bathrooms,Garages,Property Type,Property Status,Floor,Furnished
0,https://www.realethio.com/property/paster-enku...,"Paster Enkulala Fabrica, 3 bedrooms House for ...",For Sale,"50,000,000ETB",Commercial Bank Of Ethiopia (Enkulal Fabrica B...,"Paster Enkulala Fabrica, 3 bedrooms House for ...",535 m²,535 m²,3.0,1.0,8.0,House for sale,For Sale,Ground Floor,no
1,https://www.realethio.com/property/gerji-kadis...,"Gerji Kadisco, house for sale, Addis Ababa.",For Sale,"16,000,000ETB","Kadisco General Hospital, Road to Gergi Giorgi...","Gerji Kadisco, house for sale, Addis Ababa.\nT...",133 m²,133 m²,3.0,1.0,2.0,House for sale,For Sale,Ground Floor,no
2,https://www.realethio.com/property/lem-hotel-s...,"Lem Hotel Shola, G+1 Three bedrooms House for ...",For Sale,"85,000,000ETB","Lem Hotel, Equatorial Guinea Street, Megenagna...","Lem Hotel Shola, G+1 Three bedrooms House for ...",334 m²,334 m²,3.0,2.0,NaN,House for sale,For Sale,2,no
3,https://www.realethio.com/property/ayat-gift-r...,"Ayat Gift Real Estate, 2 bedrooms Apartment fo...",For Sale,"8,400,000ETB","Ayat zone 5, Bole, አዲስ አበባ / Addis Ababa, አዲስ ...","Ayat Gift Real Estate, 2 bedrooms Apartment fo...",87 m²,87 m²,2.0,1.0,1.0,Apartment for sale,For Sale,1,no
4,https://www.realethio.com/property/sarbet-two-...,"Sarbet, Two Bedroom Apartment for Sale, Addis ...",For Sale,"29,300,000ETB","AU Conference Center, Ras Lulseged Street, በግ ...","Sarbet, Two Bedroom Apartment for Sale, Addis ...",166 m²,166 m²,2.0,2.0,1.0,Apartment for sale,For Sale,16,no


In [ ]:
# Convert the text to a matrix of TF-IDF features
ethio_tfidf_vectorizer = TfidfVectorizer()
# Create a combined text column from specific columns
real_ethio_df["combined_text"] = real_ethio_df["Title"].astype(str) + " " + real_ethio_df["Type"].astype(str) + " " + real_ethio_df["Price"].astype(str) + " " + real_ethio_df["Address"].astype(str) + " " + real_ethio_df["Description"].astype(str) + " " + real_ethio_df["Bedrooms"].astype(str) + " " + real_ethio_df["Bathrooms"].astype(str) + " " + real_ethio_df["Garages"].astype(str) + " " + real_ethio_df["Property Size"].astype(str) + " " +real_ethio_df["Land Area"].astype(str)  + " " + real_ethio_df["Property Type"].astype(str)+ " " +real_ethio_df["Property Status"].astype(str) + " " + real_ethio_df["Floor"].astype(str)+ " " + real_ethio_df["Furnished"].astype(str)

ethio_tfidf_matrix = ethio_tfidf_vectorizer.fit_transform(real_ethio_df["combined_text"])

#Named Entity Recognition

In [ ]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to perform NER on the user input
def perform_ner(user_input):
    # Process the user input using spaCy
    doc = nlp(user_input)

    # Extract named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    return entities

# Example
user_input = "I want to rent a one bedroom apartment in Bole with a swimming pool and gym"
named_entities = perform_ner(user_input)

print(named_entities)


[('one', 'CARDINAL'), ('Bole', 'GPE')]


#Chatbot Integration

In [ ]:
class Chatbot:
    def __init__(self, df1, tfidf_matrix1, tfidf_vectorizer1, df2, tfidf_matrix2, tfidf_vectorizer2):
        self.df1 = df1
        self.tfidf_matrix1 = tfidf_matrix1
        self.tfidf_vectorizer1 = tfidf_vectorizer1
        self.df2 = df2
        self.tfidf_matrix2 = tfidf_matrix2
        self.tfidf_vectorizer2 = tfidf_vectorizer2
        nltk.download('punkt')
        nltk.download('stopwords')
        nltk.download('wordnet')
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()


    def preprocess_input(self, user_input):


        preprocessed_input = user_input.lower()
        preprocessed_input = preprocessed_input.replace("to buy", "for sale").replace("buy", "for sale")


        if "not furnished" in preprocessed_input or "non furnished" in preprocessed_input or "non-furnished" in preprocessed_input:
          preprocessed_input = preprocessed_input.replace("not furnished", "nonfurnished").replace("non furnished", "nonfurnished").replace("non-furnished", "nonfurnished").replace(" ", "").replace("-", "").strip()
        if "not serviced" in preprocessed_input or "non serviced" in preprocessed_input or "non-serviced" in preprocessed_input:
          preprocessed_input = preprocessed_input.replace("not serviced", "nonserviced").replace("non serviced", "nonserviced").replace("non-serviced", "nonserviced").replace(" ", "").replace("-", "").strip()
        if "not shared" in preprocessed_input or "non shared" in preprocessed_input or "non-shared" in preprocessed_input:
          preprocessed_input = preprocessed_input.replace("not shared", "nonshared").replace("non shared", "nonshared").replace("non-shared", "nonshared").replace(" ", "").replace("-", "").strip()


        # Tokenize the input
        word_tokens = word_tokenize(preprocessed_input)

        # Remove stop words
        filtered_sentence = [w for w in word_tokens if not w in self.stop_words]

        # Lemmatize the words
        lemmatized_output = [self.lemmatizer.lemmatize(w) for w in filtered_sentence]

        # Join the words back into a sentence
        preprocessed_input = ' '.join(lemmatized_output)

        return preprocessed_input

    def process_input_with_ner(self, user_input):
        named_entities = perform_ner(user_input)

        # Extract relevant entities (e.g., location, bedrooms)
        address = next((ent[0] for ent in named_entities if ent[1] == 'GPE'), None)
        bedrooms = next((ent[0] for ent in named_entities if ent[1] == 'CARDINAL'), None)

        return address, bedrooms

    def recommend_properties(self, user_input, top_n=10, country="Kenya"):
        country = country.lower()
        preprocessed_input = self.preprocess_input(user_input)


        # Determine which dataframe and TF-IDF matrix to use based on the user input
        if country == 'kenya':
            df = self.df1
            tfidf_matrix = self.tfidf_matrix1
            tfidf_vectorizer = self.tfidf_vectorizer1
        elif country == 'ethiopia':
            df = self.df2
            tfidf_matrix = self.tfidf_matrix2
            tfidf_vectorizer = self.tfidf_vectorizer2
        else:
            # Default to using df1 and its TF-IDF matrix if no specific keyword is found
            df = self.df1
            tfidf_matrix = self.tfidf_matrix1
            tfidf_vectorizer = self.tfidf_vectorizer1

        # # Apply weights to TF-IDF matrix
        # if column_weights is not None:
        #     tfidf_matrix_weighted = tfidf_matrix.copy()
        #     for column, weight in column_weights.items():
        #         # Multiply the TF-IDF values in the specified column by the weight
        #         tfidf_matrix_weighted[:, df.columns.get_loc(column)] *= weight
        # else:
        #     tfidf_matrix_weighted = tfidf_matrix


        # Preprocess the user input
        user_tfidf = tfidf_vectorizer.transform([preprocessed_input])

        # address, bedrooms = self.process_input_with_ner(user_input)

        # # Assign weights based on the extracted entities
        # weights = {
        #     'Address': 2.0,  # Higher weight for address
        #     'Bedrooms': 1.5,  # Moderate weight for bedrooms
        #     # Add more weights for other columns as needed
        # }
        # # Apply weights to TF-IDF matrix
        # if column_weights is not None:
        #     weights.update(column_weights)

        # Calculate the cosine similarity between the user input and the dataset
        cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)

        # weighted_cosine_sim = cosine_sim * np.array([weights.get(col, 1.0) for col in df.columns])

        # Flatten the matrix and get the indices of the top n highest cosine similarities
        top_indices = np.argsort(cosine_sim.flatten())[-top_n:][::-1]

        # Get the rows corresponding to the top indices
        top_rows = df.iloc[top_indices]

        # return top_rows.to_dict(orient='records')
        return top_rows

# Example usage with chat interface
bot = Chatbot(property24kenyadf, kenya_tfidf_matrix, kenya_tfidf_vectorizer, real_ethio_df, ethio_tfidf_matrix, ethio_tfidf_vectorizer)

# print("Welcome to the Property Recommender Chatbot!")
# while True:
#     user_input = input("Please enter your property preferences: ")
#     if user_input.lower() in ['exit', 'quit', 'stop']:
#         print("Exiting the chatbot. Goodbye!")
#         break
#     recommended_properties = bot.recommend_properties(user_input, top_n=5, country='Kenya')
#     print(recommended_properties)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


###GPS

In [ ]:
# # Function to geocode an address
# def geocode_address(address):
#     geolocator = Nominatim(user_agent="my_user_agent")
#     location = geolocator.geocode(address)
#     return location.latitude, location.longitude if location else (None, None)

# # Function to display a map
# def display_map(latitude, longitude, property_title):
#     property_map = folium.Map(location=[latitude, longitude], zoom_start=15)
#     folium.Marker([latitude, longitude], popup=property_title).add_to(property_map)
#     return property_map._repr_html_()

# Helper functions to safely retrieve latitude and longitude
def get_latitude(x):
    if hasattr(x, 'latitude') and (x.latitude is not None):
        return x.latitude

def get_longitude(x):
    if hasattr(x, 'longitude') and (x.longitude is not None):
        return x.longitude

# Function to geocode an address
def geocode_address(address):
    geolocator = Nominatim(user_agent="my_user_agent")
    location = geolocator.geocode(address)
    return get_latitude(location), get_longitude(location)

# Function to display a map
def display_map(latitude, longitude, property_title):
    property_map = folium.Map(location=[latitude, longitude], zoom_start=15)
    folium.Marker([latitude, longitude], popup=property_title).add_to(property_map)
    return property_map._repr_html_()


###Input Validation

In [ ]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

#Hosting the recommender to gradio

In [ ]:

# Define the function to be used by Gradio
chat_history = []
def get_recommendations(user_input, top_n, country):
    column_weights = {"Address": 2.0, "Bedrooms": 1.5, "Price": 1.5}
    if not is_english(user_input):
        return "I'm sorry, I could not understand that. Please provide a valid sentence."
    recommended_properties = bot.recommend_properties(user_input,top_n, country)

    # Add latitude and longitude columns to the DataFrame
    recommended_properties['Latitude'], recommended_properties['Longitude'] = zip(*recommended_properties['Address'].apply(geocode_address))

    # # Display a map for each recommendation and add the map HTML to the DataFrame
    # if (recommended_properties['Latitude'].notna() & recommended_properties['Longitude'].notna()).all():
    #   recommended_properties['Map'] = recommended_properties.apply(lambda row: display_map(row['Latitude'], row['Longitude'], row['Title']), axis=1)
    # else:
    #   recommended_properties['Map'] = "Unable to locate it on the map"

    # Display a map for each recommendation and add the map HTML to the DataFrame
    recommended_properties['Map'] = recommended_properties.apply(lambda row: display_map(row['Latitude'], row['Longitude'], row['Title']) if not pd.isna(row['Latitude']) and not pd.isna(row['Longitude']) else "Unable to locate it on the map", axis=1)

    # Format each recommendation as a separate string
    formatted_recommendations = []
    for index, row in recommended_properties.iterrows():
        recommendation_info = [
            f"<b>Title:</b> {row['Title']}",
            f"<b>Price:</b> {row['Price']}",
            f"<b>Type:</b> {row['Type']}",
            f"<b>Bedrooms:</b> {row['Bedrooms']}",
            f"<b>Property URL:</b> <a href='{row['Property URL']}' target='_blank'>Link</a>",
            f"<b>Description:</b> {row['Description']}",
            f"<b>Map:</b> {row['Map']}",  # Include the map HTML
            "<hr>",  # Add a horizontal line between recommendations
        ]
        formatted_recommendations.append("<br>".join(recommendation_info))

    # return "\n".join(formatted_recommendations)
    response = "\n".join(formatted_recommendations)

    # Store input and response in chat history
    chat_history.append({'user_input': user_input, 'response': response})

    # Display chat history in reverse order (latest first)
    chat_display = ""
    for entry in reversed(chat_history):
        chat_display += f"<br><b><span style='font-size:20px;'>You</span></b></br> <br>{entry['user_input']}</br> <br><b><span style='font-size:20px;'>Recommendations</span></b></br> <br>{entry['response']}</br>"

    # Add a script to clear input fields after submission
    script = """
    <script>
        document.getElementById("textbox-0").value = "";
        document.getElementById("dropdown-0").selectedIndex = 0;
        document.getElementById("dropdown-1").selectedIndex = 0;
    </script>
    """

    return chat_display + script


title = f"<h1>Your App Title</h1>"
# Create Gradio interface
iface = gr.Interface(

    fn=get_recommendations,
    inputs=[
        gr.Textbox("", label="Type in your preference"),
        gr.Dropdown(
            [5, 10, 25, 50], label="How many recommendations do you want", value=5
        ),
        gr.Dropdown(
            ["Kenya", "Ethiopia"], label="Country",value="Kenya",
        ),

    ],

    outputs=gr.HTML(),
    # live=True,  # Set live=True to display updates without reloading the page
    title="HomeFindr",
    # title = gr.HTML(title),
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6d8f3f8e7fa5c60f92.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


###Unit Testing


In [ ]:
# # Example usage with chat interface
# testbot = Chatbot(property24kenyadf, kenya_tfidf_matrix, kenya_tfidf_vectorizer, real_ethio_df, ethio_tfidf_matrix, ethio_tfidf_vectorizer)
# user_input = input("\nPlease Enter your property preferences: ")
# recommended_properties = testbot.recommend_properties(user_input,top_n=5, country="kenya")
# print(recommended_properties)


###StreamLit Application

In [ ]:
# # Define the function to be used by Streamlit
# def get_recommendations():
#     user_input = st.text_area("Type in your preference", "")
#     top_n = st.selectbox("How many recommendations do you want", [5, 10, 25, 50])
#     country = st.selectbox("Country", ["Kenya", "Ethiopia"])

#     # Process user input with NER
#     named_entities = perform_ner(user_input)
#     address = next((ent[0] for ent in named_entities if ent[1] == 'GPE'), None)
#     bedrooms = next((ent[0] for ent in named_entities if ent[1] == 'CARDINAL'), None)

#     # Get recommendations
#     recommended_properties = bot.recommend_properties(user_input, top_n, country)

#     # Add latitude and longitude columns to the DataFrame
#     recommended_properties['Latitude'], recommended_properties['Longitude'] = zip(*recommended_properties['Address'].apply(geocode_address))

#     # Display a map for each recommendation and add the map HTML to the DataFrame
#     recommended_properties['Map'] = recommended_properties.apply(lambda row: display_map(row['Latitude'], row['Longitude'], row['Title']), axis=1)


#     # Display recommendations
#     for index, row in recommended_properties.iterrows():
#         st.markdown(f"**Title:** {row['Title']}")
#         st.markdown(f"**Price:** {row['Price']}")
#         st.markdown(f"**Type:** {row['Type']}")
#         st.markdown(f"**Bedrooms:** {row['Bedrooms']}")
#         st.markdown(f"**Property URL:** [Link]({row['Property URL']})")
#         st.markdown(f"**Description:** {row['Description']}")
#         st.markdown(f"**Map:**")
#         st.markdown(folium.Map(location=[row['Latitude'], row['Longitude']], zoom_start=15)._repr_html_())

# # Run the Streamlit app
# if __name__ == "__main__":
#     st.title("HomeFindr")
#     get_recommendations()